In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install diffusers

# Understanding Stable Diffusion - Devesh Surve

## Please take a look at the blog if you want to know more - https://deveshsurve.medium.com/

# Latent Diffusion Models: Theory and Algorithmic Steps

## Introduction

Latent Diffusion Models (LDMs) are a specific type of diffusion model optimized for generating high-quality images in a more memory-efficient manner by operating in a lower-dimensional latent space. Unlike traditional diffusion models that work directly on image pixels and consume considerable memory, LDMs achieve the diffusion process in latent space, significantly reducing the memory requirements. This process involves training the model to denoise random Gaussian noise step by step to eventually produce an image.

## Main Components

There are three main components in a latent diffusion model:

1. **CLIP Text Encoder**: Converts input text into text embeddings.
2. **Variational Auto Encoder (VAE)**: Compresses and decompresses images into and from a lower-dimensional latent space.
3. **U-Net**: Predicts the noise to be removed from the noised latent representations to reconstruct the original image data.

## CLIP Text Encoder

### Basics
The CLIP Text Encoder takes text as input and produces text embeddings. These embeddings represent the text in a form that is close in the latent space to the representation of images encoded by a similar process.

### Algorithmic Steps
1. **Tokenization**: Break down the input text into sub-words or tokens and convert these into numerical representations using a lookup table.
2. **Text to Embedding Conversion**: Utilize the CLIPTextModel to convert the numerical tokens into embeddings that encapsulate the semantic meaning of the text.

### Role in Latent Diffusion
The text embeddings generated by the CLIP Text Encoder are used as one of the inputs to the U-Net model. This enables the generation of images that are semantically related to the input text.

## Variational Auto Encoder (VAE)

### Basics
The VAE consists of two main parts: an encoder and a decoder. The encoder compresses an image into a lower-dimensional latent representation, and the decoder attempts to reconstruct the image from this latent representation.

### Algorithmic Steps
1. **Encoding**: Compress an input image into a latent space representation.
2. **Decoding**: Reconstruct the image from the latent representation.

### Role in Latent Diffusion
The VAE is essential for reducing the computational load of the diffusion process. By operating in latent space, the diffusion model requires less computational power and memory, facilitating the generation of high-resolution images.

## U-Net

### Basics
The U-Net architecture takes two inputs: noisy latents and text embeddings. It outputs the predicted noise residuals to be subtracted from the noisy latents, effectively denoising them.

### Algorithmic Steps
1. **Adding Noise**: Apply a series of noise levels to the latent representations according to a predetermined schedule.
2. **Noise Prediction**: For each noise level, predict the noise present in the noisy latents.
3. **Denoising**: Subtract the predicted noise from the noisy latents to move closer to the original image representation.

### Role in Latent Diffusion
The U-Net is crucial for the iterative denoising process in latent diffusion. By gradually reducing noise from the latents, it guides the generation process to produce images that correspond to the textual description provided.

## Conclusion

Latent diffusion models represent an advanced approach in the field of generative models, especially for tasks like image synthesis. By breaking down the process into distinct components like the CLIP Text Encoder, VAE, and U-Net, LDMs manage to efficiently generate high-quality images from textual descriptions.


# Importing Required Libaries

In [ ]:
import torch, logging

## disable warnings
logging.disable(logging.WARNING)  

## Imaging  library
from PIL import Image
from torchvision import transforms as tfms

## Basic libraries
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display
import shutil
import os

## For video display
from IPython.display import HTML
from base64 import b64encode


## Import the CLIP artifacts 
from transformers import CLIPTextModel, CLIPTokenizer
from diffusers import AutoencoderKL, UNet2DConditionModel, LMSDiscreteScheduler
from IPython.display import display, clear_output
import os

In [ ]:
os.mkdir("/kaggle/working/steps2")

### Setting CPU/GPU device

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
## Initiating tokenizer and encoder.
tokenizer    = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14", torch_dtype=torch.float16)
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14", torch_dtype=torch.float16).to("cpu")

## Initiating the VAE
vae = AutoencoderKL.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="vae", torch_dtype=torch.float16).to(device)

## Initializing a scheduler and Setting number of sampling steps
scheduler = LMSDiscreteScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", num_train_timesteps=1000)
scheduler.set_timesteps(50)

## Initializing the U-Net model
unet = UNet2DConditionModel.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="unet", torch_dtype=torch.float16).to(device)

#### `load_image`

**Description:**

Loads an image from a specified path, converts it to RGB, and resizes it to a specified dimension.

**Parameters:**

- `p` (str): Path to the image file.
- `size` (tuple, optional): The dimensions to resize the image to. Default is `(512, 512)`.

**Returns:**

- `Image`: An image object in RGB format with the specified dimensions.

**Example Usage:**

```python
image = load_image("path/to/image.jpg", size=(512, 512))


In [ ]:
## Helper functions
def load_image(p):
    '''
    Function to load images from a defined path
    '''
    return Image.open(p).convert('RGB').resize((512,512))



#### `pil_to_latents`

**Description:**

Converts a PIL image to a latent representation suitable for input into a VAE model.

**Parameters:**

- `image` (PIL.Image): The image to convert.

**Returns:**

- `Tensor`: The latent representation of the image.

**Example Usage:**

```python
latents = pil_to_latents(image)


In [ ]:
def pil_to_latents(image):
    '''
    Function to convert image to latents
    '''
    init_image = tfms.ToTensor()(image).unsqueeze(0) * 2.0 - 1.0
    init_image = init_image.to(device="cuda", dtype=torch.float16) 
    init_latent_dist = vae.encode(init_image).latent_dist.sample() * 0.18215
    return init_latent_dist


#### `latents_to_pil`

**Description:**

Converts latents back into a PIL image, suitable for visualization and further processing.

**Parameters:**

- `latents` (Tensor): The latent representation to convert back to an image.

**Returns:**

- `List[Image]`: A list of image objects generated from the latent representations.

**Example Usage:**

```python
images = latents_to_pil(latents)
for img in images:
    img.show()


In [ ]:
def latents_to_pil(latents):
    '''
    Function to convert latents to images
    '''
    latents = (1 / 0.18215) * latents
    with torch.no_grad():
        image = vae.decode(latents).sample
    image = (image / 2 + 0.5).clamp(0, 1)
    image = image.detach().cpu().permute(0, 2, 3, 1).numpy()
    images = (image * 255).round().astype("uint8")
    pil_images = [Image.fromarray(image) for image in images]
    return pil_images


#### `text_enc`

**Description:**

Encodes textual prompts into embeddings using a CLIP text model.

**Parameters:**

- `prompts` (List[str]): A list of textual prompts to encode.
- `maxlen` (int, optional): Maximum length of the encoded text. Defaults to the model's maximum length.

**Returns:**

- `Tensor`: The encoded text embeddings.

**Example Usage:**

```python
embeddings = text_enc(["A clear blue sky", "The London Bridge"])


In [ ]:

def text_enc(prompts, maxlen=None):
    '''
    A function to take a texual promt and convert it into embeddings
    '''
    if maxlen is None: maxlen = tokenizer.model_max_length
    inp = tokenizer(prompts, padding="max_length", max_length=maxlen, truncation=True, return_tensors="pt") 
    return text_encoder(inp.input_ids.to("cuda"))[0].half()


#### `prompt_2_img`

**Description:**

Converts text prompts into images using a latent diffusion model.

**Parameters:**

- `prompts` (List[str]): Text prompts to convert into images.
- `g` (float): Guidance scale. Higher values enforce stronger adherence to the text prompt.
- `seed` (int): Random seed for generating images.
- `steps` (int): Number of diffusion steps.
- `dim` (int): Dimension of the generated images.
- `save_int` (bool): Whether to save intermediate images.

**Returns:**

- `List[Image]`: A list of generated image objects corresponding to the text prompts.

**Example Usage:**

```python
images = prompt_2_img(["A sunset over a mountain range", "A cat playing with a ball"], g=7.5, seed=42, steps=100, dim=512, save_int=True)
for img in images:
    img.show()


In [ ]:
def prompt_2_img(prompts, g=7.5, seed=100, steps=70, dim=512, save_int=True):
    """
    Diffusion process to convert prompt to image
    """
    
    # Defining batch size
    bs = len(prompts) 
    
    # Converting textual prompts to embedding
    text = text_enc(prompts) 
    
    # Adding an unconditional prompt , helps in the generation process
    uncond =  text_enc([""] * bs, text.shape[1])
    emb = torch.cat([uncond, text])
    
    # Setting the seed
    if seed: torch.manual_seed(seed)
    
    # Initiating random noise
    latents = torch.randn((bs, unet.in_channels, dim//8, dim//8))
    
    # Setting number of steps in scheduler
    scheduler.set_timesteps(steps)
    
    # Adding noise to the latents 
    latents = latents.to("cuda").half() * scheduler.init_noise_sigma

    print("Processing text prompts:", prompts)
    # Just before the loop starts:
    print("Visualizing initial latents...")
    latents_norm = torch.norm(latents.view(latents.shape[0], -1), dim=1).mean().item()
    print(f"Initial Latents Norm: {latents_norm}")

    # Iterating through defined steps
    for i,ts in enumerate(tqdm(scheduler.timesteps)):
        # We need to scale the i/p latents to match the variance
        inp = scheduler.scale_model_input(torch.cat([latents] * 2), ts)
        
        # Predicting noise residual using U-Net
        with torch.no_grad(): u,t = unet(inp, ts, encoder_hidden_states=emb).sample.chunk(2)
            
        # Performing Guidance
        pred = u + g*(t-u)
        
        # Conditioning  the latents
        latents = scheduler.step(pred, ts, latents).prev_sample
        
        # Inside your loop, after `latents` have been updated:
        latents_norm = torch.norm(latents.view(latents.shape[0], -1), dim=1).mean().item()
        print(f"Step {i+1}/{steps} Latents Norm: {latents_norm}")
        
        from IPython.display import display, clear_output
        if save_int and i%10==0: 
            image_path = f'steps2/la_{i:04d}.jpeg'
            latents_to_pil(latents)[0].save(image_path)
            display(latents_to_pil(latents)[0])  # Display the new image

    return latents_to_pil(latents)

# Overview — The Diffusion Process

- The stable diffusion model takes the textual input and a seed. 
- The textual input is then passed through the CLIP model to generate textual embedding of size 77x768 and the seed is used to generate Gaussian noise of size 4x64x64 which becomes the first latent image representation.
- Next, the U-Net iteratively denoises the random latent image representations while conditioning on the text embeddings. 
- The output of the U-Net is predicted noise residual, which is then used to compute conditioned latents via a scheduler algorithm. 
- This process of denoising and text conditioning is repeated N times (We will use 50) to retrieve a better latent image representation. 
- Once this process is complete, the latent image representation (4x64x64) is decoded by the VAE decoder to retrieve the final output image (3x512x512).



In [ ]:

## Initiating tokenizer and encoder.
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14", torch_dtype=torch.float16)
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14", torch_dtype=torch.float16).to("cuda")

## Initiating the VAE
vae = AutoencoderKL.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="vae", torch_dtype=torch.float16).to("cuda")

## Initializing a scheduler and Setting number of sampling steps
scheduler = LMSDiscreteScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", num_train_timesteps=1000)
scheduler.set_timesteps(50)

## Initializing the U-Net model
unet = UNet2DConditionModel.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="unet", torch_dtype=torch.float16).to("cuda")


In [ ]:
images = prompt_2_img(["A dog wearing a hat"], save_int=True)
for img in images:display(img)


In [ ]:
import os
import matplotlib.pyplot as plt

def visualize_steps(folder='/kaggle/working/steps2'):
    # Get list of all files in the folder
    files = os.listdir(folder)
    # Filter only image files
    image_files = [file for file in files if file.endswith('.jpeg')]
    num_steps = len(image_files)

    fig, axs = plt.subplots(1, num_steps, figsize=(20, 10))
    for ax, img_file in zip(axs, sorted(image_files)):  # Sort to ensure proper ordering
        img_path = os.path.join(folder, img_file)
        img = plt.imread(img_path)
        ax.imshow(img)
        ax.axis('off')  # Turn off axis numbering and ticks
        
        # Add arrows (except for the last image)
        if img_file != image_files[0]:
            ax.arrow(0.5, 0.5, 0.5, 0, head_width=0.1, head_length=0.1, fc='k', ec='k',transform=ax.transAxes, clip_on=False)

    plt.show()

# Call the function to display the visualization
visualize_steps()
